# Task C. Processing Data Stream

## Task C.1 Stream Server

In [ ]:
# Import the libraries
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

import socket
import time
import csv
import json


# Read data files and load them into memory.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

climatesDataset = []
with open('ClimateData-Part2.csv', 'r') as csvfile:
    climatesDataset = list(csv.reader(csvfile, delimiter=','))
    climatesDataset.pop(0) # Remove titles row
    
firesDataset = []
with open('FireData-Part2.csv', 'r') as csvfile:
    firesDataset = list(csv.reader(csvfile, delimiter=','))
    firesDataset.pop(0) # Remove titles row
    
    
# Timing and chunck sizes
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

timeout = 1
climateChunk = 1
fireChunk = 5


# Prepare stream server
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

host = 'localhost'
port = 50008
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((host, port))
s.listen(1)

climiteCounter = 0
fireCounter = 0
try:
    while True:
        conn, addr = s.accept()
        try:
            # Get a chunk of climates
            climates = climatesDataset[climiteCounter:(climiteCounter + climateChunk)]
            
            # Get a chunk of fires
            fires = firesDataset[fireCounter:(fireCounter + fireChunk)]
            
            # If both chunks are empty, break and close the socket.
            if not climates and not fires:
                break;
                
            # Prepare a combined package of fires and climates to send
            package = climates
            for f in fires: package.append(f)
            
            # Convert data to JSON format, encode and send data
            conn.send(json.dumps(package).encode())
            
            climiteCounter += climateChunk
            fireCounter += fireChunk
            
            # Sleep for a second before sending the next package
            time.sleep(timeout)
            
            conn.close()
        except socket.error: pass
finally:
    s.close()